In [2]:
import pandas as pd

In [ ]:
DATA_PATH = './../data/'

df = pd.read_csv(f'{DATA_PATH}household_power_consumption.txt', delimiter=';', header=0, low_memory=False)


In [18]:
df.dtypes

Date                      object
Time                      object
Global_active_power       object
Global_reactive_power     object
Voltage                   object
Global_intensity          object
Sub_metering_1            object
Sub_metering_2            object
Sub_metering_3           float64
dtype: object

In [31]:
df.isna().any()

Date                     False
Time                     False
Global_active_power      False
Global_reactive_power    False
Voltage                  False
Global_intensity         False
Sub_metering_1           False
Sub_metering_2           False
Sub_metering_3            True
dtype: bool

In [32]:
df.Sub_metering_3.isna() == True

0          False
1          False
2          False
3          False
4          False
           ...  
2075254    False
2075255    False
2075256    False
2075257    False
2075258    False
Name: Sub_metering_3, Length: 2075259, dtype: bool

In [12]:
df.columns

Index(['Date', 'Time', 'Global_active_power', 'Global_reactive_power',
       'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2',
       'Sub_metering_3'],
      dtype='object')

In [22]:
df = df.astype({'Date': 'datetime64[ns]', 'Time': 'datetime64[ns]', 'Global_active_power': 'float',
                'Global_reactive_power': 'float', 'Voltage': 'float' , 'Global_intensity': 'float',
                'Sub_metering_1': 'float', 'Sub_metering_2': 'float' , 'Sub_metering_3': 'float',})


/var/folders/4l/60lgrf9s6sqbf6lcr70gbl2c0000gn/T/ipykernel_57940/3066760354.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = df.astype({'Date': 'datetime64[ns]', 'Time': 'datetime64[ns]', 'Global_active_power': 'float',
/var/folders/4l/60lgrf9s6sqbf6lcr70gbl2c0000gn/T/ipykernel_57940/3066760354.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = df.astype({'Date': 'datetime64[ns]', 'Time': 'datetime64[ns]', 'Global_active_power': 'float',


ValueError: could not convert string to float: '?': Error while type casting for column 'Global_active_power'